# Heuristic Solver

First application of the Vehicle Routing Problem with Time Windows VRPTM for the in-house delivery service.
Leverages the Google OR-Tools.

**Future developments**
- Include a further "Orchestrator" layer to dynamically update returning riders' routes to possibly pickup deliveries from other riders just leaving the hub. This will hopefully further reduce the inactivity time of riders.
- Bring the solver into a RL environment, to take care of staging and prediction of coming orders – better to leave right now or wait another m minutes?

In [71]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

import numpy as np

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

from random import randrange

In [289]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_nodes = -1 # Takes into account first node
    nodes_visited = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            total_nodes += 1
            if manager.IndexToNode(index) != 0:
                nodes_visited.append(manager.IndexToNode(index))
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
    print('Nodes visited: {}'.format(nodes_visited))
    print('Total number of nodes: {}'.format(total_nodes))

In [340]:
def generate_data(nodes):  
    data = {}
    
    # Service area radius
    data["service_radius"] = 10
    
    # Number of nodes. Node == Order
    data["num_nodes"] = nodes
    
    # Nodes coordinates
    # First node is the HUB
    data["coord_nodes"] = [(0,0), *[(randrange(-data["service_radius"], data["service_radius"]), randrange(-data["service_radius"], data["service_radius"])) for x in range(data["num_nodes"])]]
    
    # Times matrix
    data["time_matrix"] = [[0 for x in range(data["num_nodes"]+1)] for y in range(data["num_nodes"]+1)]
    
    # Populate the time matrix
    for row, start in enumerate(data["coord_nodes"]):
        for col, end in enumerate(data["coord_nodes"]):
            x = np.array(start)
            y = np.array(end)
            distance = round(np.linalg.norm(x - y)) # Round to nearest integer
            
            if (start == (0,0)) | (end == (0,0)):
                data["time_matrix"][row][col] = distance 
            else:
                data["time_matrix"][row][col] = distance + 5
    # Node delays
    data["order_times"] = [0, *[randrange(15) for x in range(data["num_nodes"]-1)]]
    data["order_times"].sort()
    
    return data

In [341]:
def update_data(simulation_data, riders, idx, time_update, time_windows):
    data = {}
    
    # Aggregate
    data["num_vehicles"] = riders
    data["depot"] = 0
    
    # Time matrix can be extracted from the Google Distance Matrix API
    data['time_matrix'] = np.asarray(simulation_data["time_matrix"])[0:idx+2, 0:idx+2].tolist()
    
    # Time windows
    data["time_windows"] = time_windows
    for row, window in enumerate(time_windows):
        data["time_windows"][row] = (max(0, window[0] - time_update), max(0, window[1] - time_update))
    data["time_windows"].append((5,30))
    return data

In [342]:
def find_path(data):
    """Solve the VRP with time windows."""
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data["num_vehicles"], data["depot"])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        100,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    # Add penalties to drop nodes
    penalty = 100
    for node in range(1, len(data['time_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution) 
    return manager, routing, solution

In [331]:
simulation_data = generate_data()

# Optimizer is run on every order
# It outputs the riders path, cost, departure, and GREEN/RED light

# Assume a single rider
time_waited = 0
time_windows = [(0,0)]
departure = 2000
for idx, time in enumerate(simulation_data["order_times"]):
    time_waited += time
    data = update_data(simulation_data, idx, time, time_windows)
    
    # Run optimizer
    if idx > 0:
        print("\n")
        print("-------------------------------------")
        print("\n")
    print("Number of orders: ", idx+1)
    print('Time waited: {} \n'.format(time_waited))
    
    manager, routing, solution = find_path(data)
    

    # When to leave // start the path
    
    nodes_visited = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            if manager.IndexToNode(index) != 0:
                nodes_visited.append(manager.IndexToNode(index))
    
 
    

TypeError: generate_data() missing 2 required positional arguments: 'nodes' and 'riders'

In [345]:
simulation_data = generate_data(5)

In [353]:
# Assume a single rider
riders = 1
time_waited = 0
time_windows = [(0,0)]
departure = 2000
vehicles_employed = 0
for idx, time in enumerate(simulation_data["order_times"]):
    time_waited += time
    data = update_data(simulation_data, riders, idx, time, time_windows)
    
    # Run optimizer
    if idx > 0:
        print("\n")
        print("-------------------------------------")
        print("\n")
    print("Number of orders: ", idx+1)
    print('Time waited: {} \n'.format(time_waited))
    
    manager, routing, solution = find_path(data)
    

    # When to leave // start the path
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        nodes_visited = []
        while not routing.IsEnd(index):
            if manager.IndexToNode(index) != 0:
                nodes_visited.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
            
        max_time = data["time_windows"][min(nodes_visited)][1]
        print(max_time)
        
        if len(nodes_visited) >= 3:
            print("START", nodes_visited)
            vehicles_employed += 1
            
            # Update data
            # Remove nodes etc..
        if else max_time == 0:
            print(
        else:
            print("WAIT", nodes_visited)
            
        print(data)
        print(simulation_data["coord_nodes"])

Number of orders:  1
Time waited: 0 

Route for vehicle 0:
0 Time(0,0) -> 1 Time(10,10) -> 0 Time(20,20)
Time of the route: 20min

Nodes visited: [1]
Total number of nodes: 1
30
WAIT [1]
{'num_vehicles': 1, 'depot': 0, 'time_matrix': [[0, 10], [10, 5]], 'time_windows': [(0, 0), (5, 30)]}
[(0, 0), (-10, -1), (-9, 1), (-4, 5), (-4, -5), (7, 6)]


-------------------------------------


Number of orders:  2
Time waited: 0 

Route for vehicle 0:
0 Time(0,0) -> 2 Time(9,9) -> 1 Time(16,16) -> 0 Time(26,26)
Time of the route: 26min

Nodes visited: [2, 1]
Total number of nodes: 2
30
WAIT [2, 1]
{'num_vehicles': 1, 'depot': 0, 'time_matrix': [[0, 10, 9], [10, 5, 7], [9, 7, 5]], 'time_windows': [(0, 0), (5, 30), (5, 30)]}
[(0, 0), (-10, -1), (-9, 1), (-4, 5), (-4, -5), (7, 6)]


-------------------------------------


Number of orders:  3
Time waited: 2 

Route for vehicle 0:
0 Time(0,0) -> 3 Time(6,6) -> 2 Time(17,17) -> 1 Time(24,24) -> 0 Time(34,34)
Time of the route: 34min

Nodes visited: [